In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col, split

spark = SparkSession.builder.appName("StringManipulation").getOrCreate()

commentary_data = [
    (1, "Rohit Sharma hits 4 runs off Jasprit Bumrah"),
    (2, "Virat Kohli bowled by Yuzvendra Chahal for 0"),
    (3, "MS Dhoni scores 6 runs against Ravindra Jadeja"),
    (4, "KL Rahul takes 1 run from Mohammed Shami"),
    (5, "David Warner caught by Rashid Khan off his own bowling, 2 runs"),
    (6, "Shikhar Dhawan lbw b Bhuvneshwar Kumar"),
    (7, "Suryakumar Yadav edges 4 runs through slips off Trent Boult")
]

columns = ["ball_id", "commentary"]
df = spark.createDataFrame(commentary_data, columns)
df = df.withColumn("runs", 
                 regexp_extract(col("commentary"), r"(\d+)\s*run", 1).cast("int"))

df = df.withColumn("batsman", 
                 regexp_extract(col("commentary"), r"^([A-Za-z\s]+)\s+(hits|scores|takes|bowled|caught|lbw|edges)", 1))
df = df.withColumn("bowler_temp", 
                 regexp_extract(col("commentary"), r"(?:by|off|b|against)\s([A-Za-z\s]+)(?:\s*for|,|\s*$)", 1))

df = df.withColumn("bowler", 
                 regexp_extract(col("bowler_temp"), r"^([A-Za-z\s]+?)(?:\s+own|\s*$)", 1))

result_df = df.select("ball_id", "commentary", "runs", "batsman", "bowler")

spark.stop()